### Oracle Vector DB wrapped as a llama-index custom Vector Store

* inspired by: https://docs.llamaindex.ai/en/stable/examples/low_level/vector_store.html
* updated after **OCI GenAI GA**

In this **sixth demo** we show:
* how to embed a Text using OCI GenAI Embeddings (Cohere V3)
* How to query the Oracle AI Vector Store
* How to create a simplified QA retriever using LlamaIndex
* We have added phoenix traces

In [1]:
import os
import logging
import sys
import numpy as np

from typing import List, Any, Optional, Dict, Tuple
import llama_index
from llama_index.vector_stores.types import (
    VectorStore,
    VectorStoreQuery,
    VectorStoreQueryResult,
)
from llama_index import StorageContext, VectorStoreIndex, ServiceContext
from llama_index.schema import TextNode, BaseNode, Document

import oci
import ads

# only
import oracledb
from oci_utils import load_oci_config
from ads.llm import GenerativeAIEmbeddings, GenerativeAI
from oracle_vector_db import OracleVectorStore

# for observability
import phoenix as px

from config import EMBED_MODEL, PHX_PORT
from config_private import COMPARTMENT_OCID

2024-03-02 18:32:53,998 - INFO - Dataset: phoenix_dataset_32b4ed11-5303-43c0-b83e-7cdd6d7fe9ca initialized


In [2]:
# this is the endpoint after GA, for now Chicago Region
ENDPOINT = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

In [3]:
# version I'm using
print(f"oracledb version: {oracledb.__version__}")
print(f"oci version: {oci.__version__}")

oracledb version: 2.0.0.dev20231121
oci version: 2.119.1


In [4]:
# phoenix traces
os.environ["PHOENIX_PORT"] = PHX_PORT
px.launch_app()
llama_index.set_global_handler("arize_phoenix")

2024-03-02 18:32:56,227 - WARNING - OpenInferenceTraceCallbackHandler() no longer takes any arguments. The arguments provided has been ignored. For a migration guide, see https://github.com/Arize-ai/phoenix/blob/main/MIGRATION.md


🌍 To view the Phoenix app in your browser, visit http://localhost:7777/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [5]:
# for debugging
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [6]:
# setup
oci_config = load_oci_config()

# need to do this way
api_keys_config = ads.auth.api_keys(oci_config)

# english, or for other language use: multilingual

embed_model = GenerativeAIEmbeddings(
    compartment_id=COMPARTMENT_OCID,
    model=EMBED_MODEL,
    auth=api_keys_config,
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={"service_endpoint": ENDPOINT},
)

#### Using the wrapper for the DB Vector Store

In [7]:
v_store = OracleVectorStore(verbose=True)

In [8]:
question = (
    "What is JSON Relational Duality in Oracle Database 23c? Explain with details"
    # "What is Oracle Data Guard? Can it be used for Disaster recovery"
)

In [9]:
# embed the query using OCI GenAI

query_embedding = embed_model.embed_documents([question])[0]

#  wrap in llama-index
query_obj = VectorStoreQuery(query_embedding=query_embedding, similarity_top_k=6)

In [10]:
np.array(query_embedding)

array([-0.0206604 , -0.01618958, -0.02804565, ...,  0.00656509,
        0.05667114, -0.00270271])

#### Use our Vector Store DB

In [11]:
%%time

q_result = v_store.query(query_obj)

2024-03-02 18:32:58,980 - INFO - ---> Calling query on DB
2024-03-02 18:32:59,223 - INFO - SQL Query: select V.id, C.CHUNK, C.PAGE_NUM, 
                            ROUND(VECTOR_DISTANCE(V.VEC, :1, DOT), 3) as d,
                            B.NAME 
                            from VECTORS V, CHUNKS C, BOOKS B
                            where C.ID = V.ID and
                            C.BOOK_ID = B.ID
                            order by d
                            FETCH FIRST 6 ROWS ONLY
2024-03-02 18:32:59,533 - INFO - Query duration: 0.5 sec.


CPU times: user 49.4 ms, sys: 13.2 ms, total: 62.6 ms
Wall time: 570 ms


#### Displays results

In [ ]:
for n, id, sim in zip(q_result.nodes, q_result.ids, q_result.similarities):
    print(f"Doc. id: {id}")
    print(f"Similarity: {-sim}")
    print(n.text)
    print(n.metadata)
    print("")

#### Integrate in the RAG picture

In [12]:
# instantiate the client for the LLM
llm_oci = GenerativeAI(
    compartment_id=COMPARTMENT_OCID,
    max_tokens=1024,
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={"service_endpoint": ENDPOINT},
)

In [13]:
service_context = ServiceContext.from_defaults(llm=llm_oci, embed_model=embed_model)

In [14]:
index = VectorStoreIndex.from_vector_store(
    vector_store=v_store, service_context=service_context
)

In [15]:
query_engine = index.as_query_engine(similarity_top_k=5)

In [16]:
%%time

response = query_engine.query(question)

print(f"Question: {question}")
print("")
print(response.response)
print("")

2024-03-02 18:33:20,873 - INFO - ---> Calling query on DB
2024-03-02 18:33:21,045 - INFO - SQL Query: select V.id, C.CHUNK, C.PAGE_NUM, 
                            ROUND(VECTOR_DISTANCE(V.VEC, :1, DOT), 3) as d,
                            B.NAME 
                            from VECTORS V, CHUNKS C, BOOKS B
                            where C.ID = V.ID and
                            C.BOOK_ID = B.ID
                            order by d
                            FETCH FIRST 5 ROWS ONLY
2024-03-02 18:33:21,324 - INFO - Query duration: 0.4 sec.
2024-03-02 18:33:21,636 - WARNING - /Users/lsaetta/miniforge3/envs/rag_env01/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



Question: What is JSON Relational Duality in Oracle Database 23c? Explain with details

JSON Relational Duality is a feature in Oracle Database 23c that allows developers to access data stored in relational tables in the form of JSON documents. This is done by creating duality views, which are fully updatable JSON views over relational data. 

With JSON Relational Duality, developers can take advantage of the best features of both the relational and document models. The relational model is simple and powerful, while the document model offers flexibility and simplicity. 

This feature is particularly useful for building applications using relational or document models as it can help overcome the challenges that come with using these models separately. It also allows for JSON Schema-based validation, which ensures that JSON data contains mandatory fixed structures and typing. 

Would you like to know more about JSON Schema-validation or anything else regarding Oracle Database 23c feature